In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
import nibabel as nib
from nibabel.affines import apply_affine
from nibabel.streamlines import Tractogram, TckFile 
from fury import window, actor

def cargar_datos(eigenvector_path, mask_path):
    """
    Carga los datos de eigenvectores y de la máscara binaria.
    
    Parámetros:
      eigenvector_path: Ruta al archivo de eigenvectores (NIfTI).
      mask_path: Ruta al archivo de la máscara binaria (NIfTI).
      
    Retorna:
      eigen_data: Arreglo numpy de eigenvectores con forma (X,Y,Z,3).
      mask_data: Arreglo numpy binario con forma (X,Y,Z).
      affine: Matriz de transformación del espacio imagen.
    """
    eigen_img = nib.load(eigenvector_path)
    eigen_data = eigen_img.get_fdata()
    mask_img = nib.load(mask_path)
    mask_data = mask_img.get_fdata()
    affine = eigen_img.affine  # se asume que ambos archivos comparten la misma transformación
    # print(eigen_data.shape)
    # print(mask_data.shape)
    # print(eigen_data)
    # print(mask_data)
    return eigen_data, mask_data, affine
    
    
def tracking_streamline_bidireccional(eigen_data, mask, affine, seed, step_size=1.0, max_steps=1000):
    """
    Reconstruye un streamline partiendo del punto semilla en ambas direcciones. En cada paso se obtiene 
    el eigenvector principal en el voxel correspondiente. Se verifica que los nuevos puntos se encuentren 
    dentro de la máscara y de los límites de la imagen.
    
    Parámetros:
      eigen_data: Arreglo con los eigenvectores 
      mask: Arreglo binario (X,Y,Z) que indica las regiones válidas.
      affine: Matriz de transformación 
      seed: Coordenadas del punto semilla 
      step_size: Tamaño del paso en unidades espaciales.
      max_steps: Número máximo de iteraciones (por dirección).
      
    Retorna:
      streamline: Matriz numpy de puntos del streamline, de forma (N, 3).
    """
    # Primera parte: tracking en dirección forward
    streamline_forward = [np.array(seed)]
    current_point = np.array(seed)
    inv_affine = np.linalg.inv(affine)
    
    for i in range(max_steps):
        current_point = np.asarray(current_point).flatten()[:3]
        current_point_reshaped = current_point.reshape(1, 3)
        
        voxel_index = np.round(apply_affine(inv_affine, current_point_reshaped)).astype(int).squeeze()
               
        # Verificar si al menos un valor es negativo o se encuentra fuera de los limites de la mascara
        if (voxel_index < 0).any() or (voxel_index >= np.array(mask.shape)).any():
            print("Punto fuera de los límites (dirección forward).")
            break
        
        # Extraemos el primer eigenvector 
        vec_full = eigen_data[voxel_index[0], voxel_index[1], voxel_index[2], :]
        vec = vec_full[:1]  # utilizamos las primeras 3 componentes
        
        #print(vec)
        
        norm_vec = np.linalg.norm(vec)
        
        # Normalización
        direction = vec / norm_vec
        #AQUI VA LA RED 
        
        # Corroborar que el actual y el nuevo sean paralelos (no antiparalelos) via producto punto >0
        
        # Predicción
        new_point = current_point + step_size * direction
        
        streamline_forward.append(new_point)
        current_point = new_point
    
    # Segunda parte: tracking en dirección backward (sentido contrario)
    streamline_backward = []
    current_point = np.array(seed)  # Empezamos de nuevo desde la semilla
    
    for i in range(max_steps):
        current_point = np.asarray(current_point).flatten()[:3]
        current_point_reshaped = current_point.reshape(1, 3)
        
        voxel_index = np.round(apply_affine(inv_affine, current_point_reshaped)).astype(int).squeeze()
               
        # Verificar si al menos un valor es negativo o se encuentra fuera de los limites de la mascara
        if (voxel_index < 0).any() or (voxel_index >= np.array(mask.shape)).any():
            print("Punto fuera de los límites (dirección backward).")
            break
        
        # Extraemos el primer eigenvector 
        vec_full = eigen_data[voxel_index[0], voxel_index[1], voxel_index[2], :]
        vec = vec_full[:1]  # utilizamos las primeras 3 componentes
        
        #print(vec)
        
        norm_vec = np.linalg.norm(vec)
        
        # Normalización pero en dirección opuesta
        direction = -1 * (vec / norm_vec)  # Multiplicamos por -1 para ir en sentido contrario
        #AQUI VA LA RED 
        
        # Predicción en sentido opuesto
        new_point = current_point + step_size * direction
        
        streamline_backward.append(new_point)
        current_point = new_point
    
    # Unir los dos streamlines
    # Invertimos el orden de streamline_backward y eliminamos el primer punto
    # para evitar duplicar el punto semilla
    streamline_backward.reverse()
    
    # Combinamos ambos streamlines (backward + forward)
    # El punto semilla ya está incluido en streamline_forward[0]
    streamline_completo = streamline_backward + streamline_forward
    
    # Usamos vstack para asegurar regresar un arreglo numerico y evitar problemas de inhomogeneidad
    return np.vstack(streamline_completo)

def guardar_tck(streamline, affine, output_file):
    """
    Guarda el streamline en un archivo .tck.
    
    Parámetros:
      streamline: Arreglo numpy con la trayectoria (N x 3).
      affine: Matriz de transformación (voxel a RASmm).
      output_file: Ruta del archivo de salida .trk.
    """
    # Se crea un tractograma con el streamline
    tractogram = Tractogram([streamline], affine_to_rasmm=affine)
    # trk_obj = TrkFile(tractogram, header={})
    tck_obj = TckFile(tractogram, header={})
    # nib.streamlines.save(trk_obj, output_file)
    nib.streamlines.save(tck_obj, output_file)
    print(f"Streamline guardado como: {output_file}")

def visualizar_streamline(streamline):
    """
    Visualiza en 3D el streamline utilizando fury.
    
    Parámetros:
      streamline: Arreglo numpy de puntos del streamline.
    """
    scene = window.Scene()
    # Se crea un actor de línea (recibe una lista de streamlines, cada uno como un array Nx3)
    line_actor = actor.line([streamline], colors=(1, 0, 0))
    scene.add(line_actor)
    window.show(scene)
def visualizar_region_2d(nii_file, eigen_data, streamline, slice_axis='axial', slice_num=None, 
                         region_center=None, region_size=(30, 30), scale_ellipses=0.5, 
                         subsample=2, figsize=(12, 10), cmap='gray'):
    """
    Visualiza una región 2D que incluye una slice del volumen original, 
    las elipsoides de direcciones principales de difusión y la proyección del streamline.
    
    Parámetros:
        nii_file: Archivo .nii.gz cargado con nibabel o ruta al archivo
        eigen_data: Datos de eigenvectores/eigenvalores, formato (X,Y,Z,valores)
                   Asume que los primeros 3 valores son el eigenvector principal 
                   y los siguientes 3 son los eigenvalores
        streamline: Matriz numpy con las coordenadas 3D del streamline, forma (N, 3)
        slice_axis: Eje de la slice ('axial', 'sagital', 'coronal')
        slice_num: Número de slice a visualizar (si es None, se toma el centro del volumen)
        region_center: Centro de la región de interés (si es None, se usa el centro del streamline)
        region_size: Tamaño (alto, ancho) de la región a visualizar en voxels
        scale_ellipses: Factor de escala para las elipsoides
        subsample: Factor de submuestreo para no dibujar demasiadas elipsoides
        figsize: Tamaño de la figura
        cmap: Colormap para la imagen de fondo
    
    Retorna:
        fig, ax: Objetos de figura y eje para futuras modificaciones
    """
    # Cargar el volumen si se proporciona como ruta
    if isinstance(nii_file, str):
        nii_data = nib.load(nii_file)
        volume = nii_data.get_fdata()
        affine = nii_data.affine
    else:
        # Asumimos que es un objeto NIfTI ya cargado
        volume = nii_file.get_fdata()
        affine = nii_file.affine
    
    # Obtener las dimensiones del volumen
    dims = volume.shape[:3]
    
    # Determinar el eje y reordenar las coordenadas apropiadamente
    if slice_axis == 'axial':
        axis_idx = 2  # Z
        x_idx, y_idx = 0, 1  # X, Y
        xlabel, ylabel = 'X', 'Y'
    elif slice_axis == 'sagital':
        axis_idx = 0  # X
        x_idx, y_idx = 1, 2  # Y, Z
        xlabel, ylabel = 'Y', 'Z'
    elif slice_axis == 'coronal':
        axis_idx = 1  # Y
        x_idx, y_idx = 0, 2  # X, Z
        xlabel, ylabel = 'X', 'Z'
    else:
        raise ValueError("slice_axis debe ser 'axial', 'sagital' o 'coronal'")
    
    # Determinar el número de slice si no se proporciona
    if slice_num is None:
        if region_center is not None:
            slice_num = int(region_center[axis_idx])
        else:
            # Usar el centro del streamline
            streamline_center = np.mean(streamline, axis=0)
            # Convertir coordenadas del espacio físico al índice de voxel
            inv_affine = np.linalg.inv(affine)
            voxel_center = np.round(
                np.dot(inv_affine, np.append(streamline_center, 1))[:3]
            ).astype(int)
            slice_num = voxel_center[axis_idx]
    
    # Asegurarse de que el slice_num esté dentro de los límites
    slice_num = max(0, min(slice_num, dims[axis_idx] - 1))
    
    # Extraer la slice del volumen
    if axis_idx == 0:
        slice_data = volume[slice_num, :, :]
    elif axis_idx == 1:
        slice_data = volume[:, slice_num, :]
    else:  # axis_idx == 2
        slice_data = volume[:, :, slice_num]
    
    # Determinar el centro de la región si no se proporciona
    if region_center is None:
        # Proyectar el streamline a las coordenadas de voxel
        streamline_voxels = []
        for point in streamline:
            point_homogeneous = np.append(point, 1)
            voxel = np.dot(inv_affine, point_homogeneous)[:3]
            streamline_voxels.append(voxel)
        
        streamline_voxels = np.array(streamline_voxels)
        
        # Filtrar solo los puntos cercanos a la slice actual
        slice_points = []
        for point in streamline_voxels:
            if abs(point[axis_idx] - slice_num) < 2:  # Puntos cercanos a la slice
                slice_points.append(point)
        
        if len(slice_points) > 0:
            slice_points = np.array(slice_points)
            center_x = int(np.mean(slice_points[:, x_idx]))
            center_y = int(np.mean(slice_points[:, y_idx]))
        else:
            # Si no hay puntos cercanos, usar el centro de la imagen
            center_x = dims[x_idx] // 2
            center_y = dims[y_idx] // 2
    else:
        center_x = region_center[x_idx]
        center_y = region_center[y_idx]
    
    # Calcular los límites de la región
    half_width = region_size[1] // 2
    half_height = region_size[0] // 2
    
    min_x = max(0, center_x - half_width)
    max_x = min(dims[x_idx], center_x + half_width)
    min_y = max(0, center_y - half_height)
    max_y = min(dims[y_idx], center_y + half_height)
    
    # Extraer la región de interés
    if axis_idx == 0:  # Sagital
        region = slice_data[min_y:max_y, min_x:max_x]
        eigen_slice = eigen_data[slice_num, min_y:max_y, min_x:max_x, :]
    elif axis_idx == 1:  # Coronal
        region = slice_data[min_x:max_x, min_y:max_y]
        eigen_slice = eigen_data[min_x:max_x, slice_num, min_y:max_y, :]
    else:  # axis_idx == 2, Axial
        region = slice_data[min_x:max_x, min_y:max_y]
        eigen_slice = eigen_data[min_x:max_x, min_y:max_y, slice_num, :]
    
    # Crear la figura
    fig, ax = plt.subplots(figsize=figsize)
    
    # Mostrar la imagen de fondo (slice)
    ax.imshow(region.T, cmap=cmap, origin='lower')
    
    # Dibujar las elipsoides (representando las direcciones principales de difusión)
    height, width = region.shape
    for i in range(0, height, subsample):
        for j in range(0, width, subsample):
            if axis_idx == 0:  # Sagital (Y-Z)
                vec = eigen_slice[i, j, :3]  # Eigenvector principal
                vals = eigen_slice[i, j, 3:6]  # Eigenvalores
                # Proyectar el vector 3D al plano 2D (eliminar componente X)
                vec_2d = np.array([vec[1], vec[2]])  # Componentes Y, Z
            elif axis_idx == 1:  # Coronal (X-Z)
                vec = eigen_slice[i, j, :3]
                vals = eigen_slice[i, j, 3:6]
                vec_2d = np.array([vec[0], vec[2]])  # Componentes X, Z
            else:  # axis_idx == 2, Axial (X-Y)
                vec = eigen_slice[i, j, :3]
                vals = eigen_slice[i, j, 3:6]
                vec_2d = np.array([vec[0], vec[1]])  # Componentes X, Y
            
            # Normalizar el vector 2D
            norm = np.linalg.norm(vec_2d)
            if norm > 0:
                vec_2d = vec_2d / norm
                
                # Calcular el ángulo en grados para la elipse
                angle = np.degrees(np.arctan2(vec_2d[1], vec_2d[0]))
                
                # Tamaño de la elipse basado en los eigenvalores
                lambda1 = vals[0]  # Eigenvalor principal
                width_ellipse = scale_ellipses * lambda1
                height_ellipse = scale_ellipses * lambda1 / 2  # Hacerlo más elíptico
                
                # Crear y añadir la elipse
                ellipse = Ellipse((j, i), width_ellipse, height_ellipse, 
                                angle, fill=False, edgecolor='blue', linewidth=1)
                ax.add_patch(ellipse)
    
    # Proyectar y dibujar el streamline
    streamline_2d = []
    for point in streamline:
        # Convertir a coordenadas de voxel
        point_homogeneous = np.append(point, 1)
        voxel = np.dot(inv_affine, point_homogeneous)[:3]
        
        # Verificar si el punto está cerca de la slice actual
        if abs(voxel[axis_idx] - slice_num) < 2:
            if axis_idx == 0:  # Sagital
                x_coord = voxel[y_idx] - min_y
                y_coord = voxel[z_idx] - min_x
            elif axis_idx == 1:  # Coronal
                x_coord = voxel[x_idx] - min_x
                y_coord = voxel[z_idx] - min_y
            else:  # axis_idx == 2, Axial
                x_coord = voxel[x_idx] - min_x
                y_coord = voxel[y_idx] - min_y
            
            streamline_2d.append([x_coord, y_coord])
    
    # Dibujar el streamline si hay puntos
    if streamline_2d:
        streamline_2d = np.array(streamline_2d)
        ax.plot(streamline_2d[:, 0], streamline_2d[:, 1], 'r-', linewidth=2)
    
    # Ajustar los ejes y añadir etiquetas
    ax.set_xlim(0, max_x - min_x)
    ax.set_ylim(0, max_y - min_y)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    ax.set_title(f'Slice {slice_num} ({slice_axis}): Imagen, Direcciones de Difusión y Streamline')
    
    plt.tight_layout()
    return fig, ax

# Ejemplo de uso de las funciones
def ejemplo_completo(nii_file_path, seed_point, step_size=1.0, max_steps=1000):
    """
    Ejemplo completo que muestra cómo usar las funciones de tracking y visualización.
    
    Parámetros:
        nii_file_path: Ruta al archivo .nii.gz
        seed_point: Punto semilla para el tracking
        step_size: Tamaño del paso para el tracking
        max_steps: Número máximo de pasos para el tracking
    """
    # Cargar datos
    img = nib.load(nii_file_path)
    data = img.get_fdata()
    affine = img.affine
    
    # Cargar o calcular los eigenvectores/eigenvalores del tensor de difusión
    # Nota: Este es un ejemplo simplificado. En un caso real, necesitarías calcular
    # o cargar estos datos a partir de tus imágenes de DTI.
    # eigen_data tiene forma (X, Y, Z, valores) donde valores incluye
    # [vec1_x, vec1_y, vec1_z, lambda1, lambda2, lambda3]
    eigen_data = np.zeros(data.shape + (6,))  # Placeholder
    
    # Crear una máscara (ejemplo simple)
    mask = data > 0  # Simplificación
    
    # Generar el streamline
    streamline = tracking_streamline_bidireccional(eigen_data, mask, affine, seed_point, 
                                                  step_size, max_steps)
    
    # Visualizar
    fig, ax = visualizar_region_2d(
        nii_file=img,
        eigen_data=eigen_data,
        streamline=streamline,
        slice_axis='axial',  # Puedes cambiar a 'sagital' o 'coronal'
        slice_num=None,  # Automáticamente determinado
        region_size=(40, 40),
        scale_ellipses=0.8,
        subsample=3
    )
    
    plt.show()
    return streamline, fig, ax

In [ ]:


ejemplo_completo("NoArtifacts_Relaxation.nii.gz", )